In [1]:
import numpy as np
import torch

In [2]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

In [3]:
# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

In [4]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [5]:
# Weights and biases
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)
print(w)
print(b)

tensor([[ 0.1283, -0.3646,  0.3990],
        [-0.3371,  1.0491,  1.9137]], requires_grad=True)
tensor([-0.2584,  0.6869], requires_grad=True)


`torch.randn` creates a tensor with the given shape, with elements picked randomly from a [normal distribution](https://en.wikipedia.org/wiki/Normal_distribution) with mean 0 and standard deviation 1.

Our *model* is simply a function that performs a matrix multiplication of the `inputs` and the weights `w` (transposed) and adds the bias `b` (replicated for each observation).

![matrix-mult](https://i.imgur.com/WGXLFvA.png)

We can define the model as follows:

In [6]:
def model(x):
    return x @ w.t() + b

`@` represents matrix multiplication in PyTorch, and the `.t` method returns the transpose of a tensor.

The matrix obtained by passing the input data into the model is a set of predictions for the target variables.

In [7]:
preds = model(inputs)
print(preds)

tensor([[  1.8416, 128.6595],
        [  4.8750, 184.8111],
        [-14.8038, 222.9380],
        [ 11.9198,  82.2219],
        [  1.5288, 212.1027]], grad_fn=<AddBackward0>)


In [8]:
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [9]:
def mse(t1, t2):
    diff = t1 - t2
    return torch.sum(diff * diff) / diff.numel() # diff.numel number of elements in the matrix

In [10]:
# Compute Loss
loss = mse(preds, targets)
print(loss)

tensor(6629.6836, grad_fn=<DivBackward0>)


In [11]:
loss.backward()

In [12]:
print(w)
print(w.grad)

tensor([[ 0.1283, -0.3646,  0.3990],
        [-0.3371,  1.0491,  1.9137]], requires_grad=True)
tensor([[-6110.3125, -7686.4023, -4487.4766],
        [ 6154.0562,  6847.9307,  4258.6143]])


In [13]:
print(b)
print(b.grad)

tensor([-0.2584,  0.6869], requires_grad=True)
tensor([-75.1277,  74.1466])


Before we proceed, we reset the gradients to zero by invoking the `.zero_()` method. We need to do this because PyTorch accumulates gradients. Otherwise, the next time we invoke `.backward` on the loss, the new gradient values are added to the existing gradients, which may lead to unexpected results.

In [14]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


In [15]:
# Generate predictions
preds = model(inputs)
print(preds)

tensor([[  1.8416, 128.6595],
        [  4.8750, 184.8111],
        [-14.8038, 222.9380],
        [ 11.9198,  82.2219],
        [  1.5288, 212.1027]], grad_fn=<AddBackward0>)


In [16]:
# Calculate the loss
loss = mse(preds, targets)
print(loss)

tensor(6629.6836, grad_fn=<DivBackward0>)


In [17]:
# Compute gradients
loss.backward()
print(w.grad)
print(b.grad)

tensor([[-6110.3125, -7686.4023, -4487.4766],
        [ 6154.0562,  6847.9307,  4258.6143]])
tensor([-75.1277,  74.1466])


In [18]:
# Adjust weights & reset gradients
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

In [19]:
print(w)
print(b)

tensor([[ 0.1895, -0.2877,  0.4439],
        [-0.3986,  0.9807,  1.8711]], requires_grad=True)
tensor([-0.2576,  0.6862], requires_grad=True)


In [20]:
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(4630.9961, grad_fn=<DivBackward0>)


In [21]:
# Train for 100 epochs
for i in range(1000):
    preds = model(inputs)
    loss = mse(preds, targets)
    loss.backward()
    with torch.no_grad():
        w -= w.grad * 1e-5
        b -= b.grad * 1e-5
        w.grad.zero_()
        b.grad.zero_()
    

In [22]:
preds = model(inputs)
loss = mse(preds, targets)
print(loss)

tensor(5.5103, grad_fn=<DivBackward0>)


In [23]:
print(preds)
print(targets)

tensor([[ 57.1200,  70.1545],
        [ 82.3972, 102.8212],
        [118.3321, 128.3170],
        [ 21.0955,  35.6814],
        [102.1917, 123.7927]], grad_fn=<AddBackward0>)
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [24]:
import torch.nn as nn

In [25]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')

# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [26]:
from torch.utils.data import TensorDataset

In [27]:
train_ds = TensorDataset(inputs, targets)
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]),
 tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [28]:
from torch.utils.data import DataLoader

In [30]:
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [33]:
for xb, yb in train_dl:
    print("batch")
    print(xb)
    print(yb)
    break

batch
tensor([[102.,  43.,  37.],
        [ 91.,  87.,  65.],
        [ 92.,  87.,  64.],
        [103.,  43.,  36.],
        [ 74.,  66.,  43.]])
tensor([[ 22.,  37.],
        [ 80., 102.],
        [ 82., 100.],
        [ 20.,  38.],
        [ 57.,  69.]])


In [34]:
model = nn.Linear(3, 2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[0.2077, 0.5385, 0.4530],
        [0.2214, 0.3138, 0.4651]], requires_grad=True)
Parameter containing:
tensor([-0.2741, -0.4564], requires_grad=True)


In [35]:
list(model.parameters())

[Parameter containing:
 tensor([[0.2077, 0.5385, 0.4530],
         [0.2214, 0.3138, 0.4651]], requires_grad=True),
 Parameter containing:
 tensor([-0.2741, -0.4564], requires_grad=True)]

In [36]:
preds = model(inputs)
preds

tensor([[ 70.4457,  56.7225],
        [ 95.0051,  77.0624],
        [116.2266,  87.8190],
        [ 60.8285,  52.8213],
        [ 97.4608,  77.4931],
        [ 70.1149,  56.6301],
        [ 94.9196,  77.2137],
        [116.8873,  88.5054],
        [ 61.1592,  52.9137],
        [ 97.7060,  77.7368],
        [ 70.3602,  56.8738],
        [ 94.6744,  76.9700],
        [116.3121,  87.6677],
        [ 60.5832,  52.5776],
        [ 97.7915,  77.5855]], grad_fn=<AddmmBackward>)

In [37]:
import torch.nn.functional as F # different loss function

In [38]:
loss_fn = F.mse_loss

In [45]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(670.2604, grad_fn=<MseLossBackward>)


In [46]:
# Define optimizer
opt = torch.optim.SGD(model.parameters(), lr=1e-5)

In [53]:
def fit(num_epochs, model, loss_fn, opt):
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        # Train with batches of data
        for xb, yb in train_dl:
            # 1. Generate Prediction
            pred = model(xb)
            # 2. Calculate Loss
            loss = loss_fn(pred, yb)
            # 3. Compute gradients
            loss.backward()
            # 4. Update parameters using gradients
            opt.step()
            # 5. Reset the gradients to zero
            opt.zero_grad()
        
        if (epoch+1) %10 == 0:
            print(f"[{epoch+1}/{num_epochs}], loss: {loss.item()}") # loss.item value of loss

In [54]:
fit(100, model, loss_fn, opt)

[10/100], loss: 8.892532348632812
[20/100], loss: 9.572853088378906
[30/100], loss: 6.274220943450928
[40/100], loss: 7.473645210266113
[50/100], loss: 4.538607597351074
[60/100], loss: 4.150024890899658
[70/100], loss: 3.0569875240325928
[80/100], loss: 3.663893938064575
[90/100], loss: 5.8017706871032715
[100/100], loss: 2.264270305633545
